<a href="https://colab.research.google.com/github/dohyun93/hongong_mldl/blob/main/8_1_%ED%95%A9%EC%84%B1%EA%B3%B1_%EC%8B%A0%EA%B2%BD%EB%A7%9D%EC%9D%98_%EA%B5%AC%EC%84%B1_%EC%9A%94%EC%86%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 합성곱 신경망 (Convolutional Neural Network)

> 완전연결 신경망에서는 입력이 10개, 뉴런이 100개일 경우, 출력도 100개였다. 가중치는 10*1000+절편개수만큼 존재했다.

> 합성곱 신경망에서는 이미지 처리 분야에서 뛰어난 성능을 보이는 인공신경망으로, 2차원의 필터에 있는 각 파라미터 (커널)를 왼쪽에서 오른쪽으로, 위에서 아래로 입력에 곱한 출력을 2차원의 피쳐맵으로 만든다.

> 이런 필터(=뉴런, 커널)는 여러개를 사용할 수 있다.

> 합성곱 신경망은 입력데이터 대비 매우 작은 필터를 합성곱하여 유용한 특성만 출력되게 한다.

In [1]:
# 케라스 합성곱 층

from tensorflow import keras
keras.layers.Conv2D(10, kernel_size=(3, 3), activation='relu')
# 10: 필터의 개수.
# kernel_size: 필터에 사용할 커널의 크기. 일반적으로 (3,3), (5,5)가 권장된다.
# 피쳐맵은 합성곱이후 활성화함수를 거친 결과이다.

# 합성곱 신경망 1개 이상이 들어간 신경망을 합성곱 신경망이라고 부른다.
# 꼭 합성곱 층만 있어야 합성곱 신경망이라고 부르는것은 아니다.

---

> 패딩 (Padding) : 입력에 필터를 합성곱하면 피쳐맵의 크기는 입력보다 작아진다. 이를 ```valid padding``` 이라고 한다. 반면, 입력의 가장자리에 가상으로 0으로 채워진 픽셀들을 추가하고, 이를 필터로 합성곱 하여 (가상의 값이 없는 원본)입력과 크기가 동일한 피쳐맵을 만드는 것을 ```same padding``` 이라고 한다. 일반적으로 same padding을 많이 사용하는데, 그 이유는 필터가 합성곱하여 피쳐맵에 반영시킬 때, 입력의 각 픽셀이 같은 비율로 반영되기 때문에, 가장자리가 무시되고 가운데 부분의 특성이 전달되는 밸리드 패딩이 가장자리의 중요한 특성을 잃어버리는 단점을 보완할 수 있다.

---

> 스트라이드 (Stride) : 입력에 필터를 합성곱할 때, 이동하는 칸을 의미한다. 일반적으로 1이 사용된다. (1, 1)처럼 튜플로 전달될 수 있으나, 이 값이 각각 다른 경우는 거의 없고, 1보다 큰 스트라이드를 사용하는 경우도 거의 없다.

---

> 풀링 (Pooling) : 피쳐맵의 크기를 줄여주는 역할을 한다 (피쳐맵의 개수는 유지). (2, 2) 크기로 풀링했다고 할 때, (4, 4)의 피쳐맵은 (2, 2) 크기로 줄어든다. ```풀링 과정에서 합성곱을 할 때, 필터를 이전 합성곱의 영역과 '겹치지 않게' 합성곱을 하기 때문이다.``` 풀링을 할 때, 풀링 크기에 속하는 피쳐맵의 요소들 가운데 최대값만 남기는 것을 '최대 풀링(max pooling)'이라고 하며, 평균값을 남기는 것을 '평균 풀링'(average pooling) 이라고 한다. 평균 풀링은 평균할 때 피쳐맵의 중요 특성이 희석될 수 있기 때문에, 보통 최대 풀링이 더 많이 사용된다. strides 크기는 기본값으로 풀링의 크기이므로 따로 지정하지 않아도 되며, padding 기본값은 valid로, 패딩을 하지 않는다. 이 두 매개변수는 기본값에서 거의 바뀌지 않는다. ```풀링을 사용하는 이유는, 풀링 층에서 풀링을 사용하여 피쳐맵의 크기를 줄이는 것이 합성곱 층에서 stride를 1보다 크게 하여 피쳐맵의 크기를 줄이는 것보다 경험적으로 더 나은 성능을 보이기 때문이다.```

---

In [2]:
# 합성곱 신경망의 전체 구조.

# 앞서 살펴본 입력은 (4, 4) 처럼 2차원 크기의 배열이었다.
# 그러나 케라스는 내부적으로 RGB에 대한 이미지처리를 하도록 구현되어 있다.
# 따라서 (4, 4) 이미지는 실제로 (너비, 높이, 깊이) = (4, 4, 3) 라는 형태로 입력된다. (흑백은 (4, 4, 1))

# 이 때, 필터 크기가 (3, 3) 이었다면 깊이 차원이 추가된 필터인 (3, 3, 3)이 사용된다.
# 즉, 커널의 깊이 차원은 입력의 깊이 차원과 항상 같다.

# 1번째 합성곱 신경망의 피쳐맵 : (4, 4, 5)
# 2번째 합성곱 신경망의 커널크기가 너비, 높이가 (3, 3) 일 경우 입력의 깊이와 동일하므로 (3, 3, 5)가 된다.
# 이런 필터가 10개일 경우, 2번째 합성곱 신경망의 피쳐맵은 (2, 2, 10)이 된다.

# 이렇게 합성곱 신경망이 깊어질 수록, 너비, 높이는 점점 줄어들고, 깊이는 점점 깊어지는 것이 특징이다.
# 합성곱 신경망에서 '필터'는 이미지에 있는 어떤 특징을 찾는다고 생각할 수 있다.
# 처음에는 기본적인 특징(직선, 곡선 등)을 찾고, 층이 깊어질수록 다양하고 구체적인 특징을 감지할 수 있도록 필터의 개수를 늘린다.
# 또, 어떤 특징이 이미지의 어느 위치에 높이더라도 쉽게 감지할 수 있도록 너비와 높이 차원을 점점 줄여나간다.

# 마지막에 출력층 전에 특성 맵을 모두 펼쳐서 밀집층의 입력으로 사용한다.